# 🎬 LOCOMOT.IO Automatic Clip Generator

Runs headless game simulation, detects exciting moments, and renders promotional clips sorted by excitement.

**Features:**
- Headless Python game simulation
- AI trains with neural network brains
- Automatic excitement detection (kills, streaks, close calls)
- Renders clips with smooth animations
- Sorts by excitement score for best ad content

In [ ]:
# Install dependencies
!pip install -q numpy pillow imageio imageio-ffmpeg

In [ ]:
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import imageio
from collections import deque
import random
import math
from dataclasses import dataclass, field
from typing import List, Tuple, Optional, Dict
import time
from IPython.display import HTML, display
import base64
import os

In [ ]:
# === GAME CONSTANTS ===
WORLD_W, WORLD_H = 80, 60
GRID = 12
CANVAS_W, CANVAS_H = 480, 360  # Render size
FPS = 30
CLIP_SECONDS = 6
BUFFER_FRAMES = FPS * CLIP_SECONDS

# Colors (RGB)
COLORS = {
    'bg': (17, 17, 17),
    'grid': (34, 34, 34),
    'player': (0, 255, 0),
    'enemy': (255, 0, 255),
    'food': (255, 255, 0),
    'bullet': (255, 100, 100),
    'text': (0, 255, 0),
    'kill_flash': (255, 50, 50),
}

TRAIN_COLORS = [
    (255, 0, 255),   # Magenta
    (0, 175, 255),   # Blue
    (255, 85, 85),   # Red
    (255, 200, 0),   # Gold
    (0, 255, 200),   # Cyan
    (255, 100, 200), # Pink
    (200, 100, 255), # Purple
    (100, 255, 100), # Light green
]

In [ ]:
# === TRAIN CLASS ===
@dataclass
class Segment:
    x: int
    y: int
    hp: int = 100

@dataclass 
class Train:
    id: int
    name: str
    segments: List[Segment] = field(default_factory=list)
    color: Tuple[int, int, int] = (255, 0, 255)
    direction: Tuple[int, int] = (1, 0)
    alive: bool = True
    kills: int = 0
    
    # AI brain weights (simple neural net)
    brain: np.ndarray = None
    
    def __post_init__(self):
        if self.brain is None:
            # Simple brain: 8 inputs -> 8 hidden -> 3 outputs (left, straight, right)
            self.brain = np.random.randn(8, 8) * 0.5
            self.brain2 = np.random.randn(8, 3) * 0.5
    
    @property
    def head(self):
        return self.segments[0] if self.segments else None
    
    @property
    def length(self):
        return len(self.segments)
    
    def get_vision(self, game) -> np.ndarray:
        """Get 8-direction vision: distance to wall/enemy/food"""
        if not self.head:
            return np.zeros(8)
        
        vision = []
        directions = [(0,-1), (1,-1), (1,0), (1,1), (0,1), (-1,1), (-1,0), (-1,-1)]
        
        for dx, dy in directions:
            dist = 0
            x, y = self.head.x, self.head.y
            found = False
            
            while dist < 20 and not found:
                x = (x + dx) % WORLD_W
                y = (y + dy) % WORLD_H
                dist += 1
                
                # Check for other trains
                for train in game.trains:
                    if train.id != self.id and train.alive:
                        for seg in train.segments:
                            if seg.x == x and seg.y == y:
                                found = True
                                break
                    if found:
                        break
            
            vision.append(1.0 - dist / 20.0)  # Closer = higher value
        
        return np.array(vision)
    
    def think(self, game) -> int:
        """Returns 0=left, 1=straight, 2=right"""
        vision = self.get_vision(game)
        hidden = np.tanh(vision @ self.brain)
        output = hidden @ self.brain2
        return int(np.argmax(output))
    
    def turn(self, decision: int):
        """Turn based on decision: 0=left, 1=straight, 2=right"""
        dx, dy = self.direction
        if decision == 0:  # Left
            self.direction = (dy, -dx)
        elif decision == 2:  # Right
            self.direction = (-dy, dx)
        # decision == 1 is straight, no change
    
    def move(self):
        if not self.alive or not self.head:
            return
        
        new_x = (self.head.x + self.direction[0]) % WORLD_W
        new_y = (self.head.y + self.direction[1]) % WORLD_H
        
        # Move segments
        for i in range(len(self.segments) - 1, 0, -1):
            self.segments[i].x = self.segments[i-1].x
            self.segments[i].y = self.segments[i-1].y
        
        self.head.x = new_x
        self.head.y = new_y
    
    def grow(self, amount: int = 1):
        for _ in range(amount):
            tail = self.segments[-1]
            self.segments.append(Segment(tail.x, tail.y))

In [ ]:
# === EXCITING EVENT TRACKING ===
@dataclass
class ExcitingEvent:
    frame: int
    event_type: str  # 'kill', 'streak', 'close_call', 'leader_change'
    score: float
    details: Dict = field(default_factory=dict)

class ExcitementTracker:
    def __init__(self):
        self.events: List[ExcitingEvent] = []
        self.kill_times: Dict[int, List[int]] = {}  # train_id -> [frame, frame, ...]
        self.leader_id: Optional[int] = None
    
    def on_kill(self, frame: int, killer: Train, victim: Train):
        # Track kill streak
        if killer.id not in self.kill_times:
            self.kill_times[killer.id] = []
        self.kill_times[killer.id].append(frame)
        
        # Count recent kills (within 3 seconds)
        recent = [f for f in self.kill_times[killer.id] if frame - f < FPS * 3]
        streak = len(recent)
        
        # Score: base kill + streak bonus + length differential
        length_diff = killer.length - victim.length
        score = 50 + (streak - 1) * 30 + max(0, -length_diff) * 5  # Underdog bonus
        
        event_type = 'streak' if streak >= 2 else 'kill'
        self.events.append(ExcitingEvent(
            frame=frame,
            event_type=event_type,
            score=score,
            details={'killer': killer.name, 'victim': victim.name, 'streak': streak}
        ))
        print(f"  🔥 {event_type.upper()}: {killer.name} eliminated {victim.name} (streak: {streak}, score: {score})")
    
    def on_close_call(self, frame: int, train: Train, distance: float):
        # Near miss - almost died
        score = 30 + (5 - distance) * 10
        self.events.append(ExcitingEvent(
            frame=frame,
            event_type='close_call',
            score=score,
            details={'train': train.name, 'distance': distance}
        ))
    
    def on_leader_change(self, frame: int, new_leader: Train, old_leader: Optional[Train]):
        if self.leader_id != new_leader.id:
            score = 40
            self.events.append(ExcitingEvent(
                frame=frame,
                event_type='leader_change',
                score=score,
                details={'new_leader': new_leader.name, 'length': new_leader.length}
            ))
            self.leader_id = new_leader.id
    
    def get_best_moments(self, n: int = 5) -> List[ExcitingEvent]:
        """Return top N exciting moments by score"""
        return sorted(self.events, key=lambda e: e.score, reverse=True)[:n]

In [ ]:
# === HEADLESS GAME ===
class HeadlessGame:
    def __init__(self, num_trains: int = 8):
        self.trains: List[Train] = []
        self.food: List[Tuple[int, int]] = []
        self.particles: List[Dict] = []
        self.frame = 0
        self.tracker = ExcitementTracker()
        self.frame_buffer = deque(maxlen=BUFFER_FRAMES)
        
        # Initialize trains
        names = ['ShadowX', 'NightWolf', 'BlazeMaster', 'IceKing', 'ThunderBolt', 
                 'DarkPhoenix', 'StormRider', 'CyberNinja', 'GhostTrain', 'IronHorse']
        
        for i in range(num_trains):
            x = random.randint(10, WORLD_W - 10)
            y = random.randint(10, WORLD_H - 10)
            train = Train(
                id=i,
                name=names[i % len(names)] + str(random.randint(10, 99)),
                segments=[Segment(x, y), Segment(x-1, y), Segment(x-2, y)],
                color=TRAIN_COLORS[i % len(TRAIN_COLORS)],
                direction=random.choice([(1,0), (-1,0), (0,1), (0,-1)])
            )
            self.trains.append(train)
        
        # Spawn food
        for _ in range(30):
            self.spawn_food()
    
    def spawn_food(self):
        x = random.randint(0, WORLD_W - 1)
        y = random.randint(0, WORLD_H - 1)
        self.food.append((x, y))
    
    def respawn_train(self, train: Train):
        x = random.randint(10, WORLD_W - 10)
        y = random.randint(10, WORLD_H - 10)
        train.segments = [Segment(x, y), Segment(x-1, y), Segment(x-2, y)]
        train.direction = random.choice([(1,0), (-1,0), (0,1), (0,-1)])
        train.alive = True
    
    def step(self):
        self.frame += 1
        alive_trains = [t for t in self.trains if t.alive]
        
        # AI decisions and movement
        for train in alive_trains:
            decision = train.think(self)
            train.turn(decision)
            train.move()
        
        # Food collection
        for train in alive_trains:
            head = train.head
            for i, (fx, fy) in enumerate(self.food):
                if head.x == fx and head.y == fy:
                    train.grow(2)
                    self.food.pop(i)
                    self.spawn_food()
                    break
        
        # Collision detection
        for train in alive_trains:
            head = train.head
            
            for other in alive_trains:
                if other.id == train.id:
                    continue
                
                for seg in other.segments:
                    if head.x == seg.x and head.y == seg.y:
                        # Collision!
                        if train.length > other.length:
                            # Train wins
                            train.kills += 1
                            train.grow(other.length // 2)
                            self.add_death_particles(other)
                            self.tracker.on_kill(self.frame, train, other)
                            other.alive = False
                        elif other.length > train.length:
                            # Other wins
                            other.kills += 1
                            other.grow(train.length // 2)
                            self.add_death_particles(train)
                            self.tracker.on_kill(self.frame, other, train)
                            train.alive = False
                        else:
                            # Both die
                            self.add_death_particles(train)
                            self.add_death_particles(other)
                            train.alive = False
                            other.alive = False
                        break
                if not train.alive:
                    break
        
        # Respawn dead trains
        for train in self.trains:
            if not train.alive:
                self.respawn_train(train)
        
        # Track leader
        if alive_trains:
            leader = max(alive_trains, key=lambda t: t.length)
            old_leader = next((t for t in self.trains if t.id == self.tracker.leader_id), None)
            self.tracker.on_leader_change(self.frame, leader, old_leader)
        
        # Update particles
        self.particles = [p for p in self.particles if p['life'] > 0]
        for p in self.particles:
            p['x'] += p['vx']
            p['y'] += p['vy']
            p['life'] -= 1
    
    def add_death_particles(self, train: Train):
        for seg in train.segments:
            for _ in range(3):
                self.particles.append({
                    'x': seg.x * GRID,
                    'y': seg.y * GRID,
                    'vx': random.uniform(-3, 3),
                    'vy': random.uniform(-3, 3),
                    'life': 20,
                    'color': train.color
                })

In [ ]:
# === RENDERER ===
class GameRenderer:
    def __init__(self):
        self.scale = CANVAS_W / (WORLD_W * GRID)
    
    def render_frame(self, game: HeadlessGame, camera_target: Optional[Train] = None) -> Image.Image:
        img = Image.new('RGB', (CANVAS_W, CANVAS_H), COLORS['bg'])
        draw = ImageDraw.Draw(img)
        
        # Camera offset (follow target or center)
        if camera_target and camera_target.head:
            cam_x = camera_target.head.x * GRID - CANVAS_W // 2
            cam_y = camera_target.head.y * GRID - CANVAS_H // 2
        else:
            cam_x = (WORLD_W * GRID - CANVAS_W) // 2
            cam_y = (WORLD_H * GRID - CANVAS_H) // 2
        
        # Draw grid
        for x in range(0, WORLD_W * GRID, GRID):
            sx = x - cam_x
            if 0 <= sx < CANVAS_W:
                draw.line([(sx, 0), (sx, CANVAS_H)], fill=COLORS['grid'])
        for y in range(0, WORLD_H * GRID, GRID):
            sy = y - cam_y
            if 0 <= sy < CANVAS_H:
                draw.line([(0, sy), (CANVAS_W, sy)], fill=COLORS['grid'])
        
        # Draw food
        for fx, fy in game.food:
            sx = fx * GRID - cam_x + GRID // 2
            sy = fy * GRID - cam_y + GRID // 2
            if 0 <= sx < CANVAS_W and 0 <= sy < CANVAS_H:
                r = 4
                draw.ellipse([sx-r, sy-r, sx+r, sy+r], fill=COLORS['food'])
        
        # Draw trains
        for train in game.trains:
            if not train.alive:
                continue
            
            for i, seg in enumerate(train.segments):
                sx = seg.x * GRID - cam_x
                sy = seg.y * GRID - cam_y
                
                if -GRID <= sx < CANVAS_W + GRID and -GRID <= sy < CANVAS_H + GRID:
                    # Dim color for body, bright for head
                    if i == 0:
                        color = train.color
                        size = GRID - 2
                    else:
                        color = tuple(c // 2 for c in train.color)
                        size = GRID - 4
                    
                    offset = (GRID - size) // 2
                    draw.rectangle([sx + offset, sy + offset, sx + offset + size, sy + offset + size], fill=color)
        
        # Draw particles
        for p in game.particles:
            sx = int(p['x'] - cam_x)
            sy = int(p['y'] - cam_y)
            if 0 <= sx < CANVAS_W and 0 <= sy < CANVAS_H:
                alpha = p['life'] / 20
                color = tuple(int(c * alpha) for c in p['color'])
                r = 2
                draw.ellipse([sx-r, sy-r, sx+r, sy+r], fill=color)
        
        # Draw leaderboard
        sorted_trains = sorted(game.trains, key=lambda t: t.length, reverse=True)[:5]
        y_pos = 10
        for i, train in enumerate(sorted_trains):
            text = f"{i+1}. {train.name}: {train.length}"
            # Simple text (no custom font in Colab)
            draw.text((CANVAS_W - 120, y_pos), text, fill=train.color)
            y_pos += 15
        
        return img
    
    def render_clip(self, frames: List[Image.Image], filename: str):
        """Render frames to video file"""
        with imageio.get_writer(filename, fps=FPS, quality=8) as writer:
            for frame in frames:
                writer.append_data(np.array(frame))
        print(f"  ✅ Saved: {filename}")

In [ ]:
# === CLIP EXTRACTOR ===
class ClipExtractor:
    def __init__(self, game: HeadlessGame, renderer: GameRenderer):
        self.game = game
        self.renderer = renderer
        self.frame_buffer: deque = deque(maxlen=BUFFER_FRAMES)
    
    def run_simulation(self, total_frames: int = 3000, progress_interval: int = 500):
        """Run simulation and collect frames"""
        print(f"🎮 Running simulation for {total_frames} frames ({total_frames/FPS:.1f} seconds)...")
        
        all_frames = []
        
        for i in range(total_frames):
            self.game.step()
            
            # Find current leader for camera
            leader = max(self.game.trains, key=lambda t: t.length if t.alive else 0)
            
            # Render frame
            frame = self.renderer.render_frame(self.game, camera_target=leader)
            all_frames.append((self.game.frame, frame))
            
            if i > 0 and i % progress_interval == 0:
                print(f"  Frame {i}/{total_frames} ({i*100/total_frames:.0f}%)")
        
        print(f"✅ Simulation complete! Found {len(self.game.tracker.events)} exciting events.")
        return all_frames
    
    def extract_clips(self, all_frames: List, num_clips: int = 5) -> List[Dict]:
        """Extract best clips around exciting events"""
        best_events = self.game.tracker.get_best_moments(num_clips)
        
        print(f"\n🎬 Extracting {len(best_events)} clips...")
        
        clips = []
        frames_dict = {f[0]: f[1] for f in all_frames}
        
        for i, event in enumerate(best_events):
            # Get frames around event (3 sec before, 3 sec after)
            start_frame = max(1, event.frame - FPS * 3)
            end_frame = min(max(frames_dict.keys()), event.frame + FPS * 3)
            
            clip_frames = []
            for f in range(start_frame, end_frame):
                if f in frames_dict:
                    clip_frames.append(frames_dict[f])
            
            if len(clip_frames) > FPS:  # At least 1 second
                clips.append({
                    'event': event,
                    'frames': clip_frames,
                    'rank': i + 1
                })
                print(f"  #{i+1} {event.event_type.upper()} (score: {event.score:.0f}) - {event.details}")
        
        return clips
    
    def save_clips(self, clips: List[Dict], output_dir: str = 'clips'):
        """Save clips as video files"""
        os.makedirs(output_dir, exist_ok=True)
        
        print(f"\n💾 Saving {len(clips)} clips...")
        
        saved_files = []
        for clip in clips:
            event = clip['event']
            filename = f"{output_dir}/clip_{clip['rank']:02d}_{event.event_type}_{event.score:.0f}pts.mp4"
            self.renderer.render_clip(clip['frames'], filename)
            saved_files.append(filename)
        
        return saved_files

In [ ]:
# === RUN THE PIPELINE ===
print("🚂 LOCOMOT.IO Clip Generator")
print("=" * 40)

# Initialize
game = HeadlessGame(num_trains=8)
renderer = GameRenderer()
extractor = ClipExtractor(game, renderer)

# Run simulation (100 seconds of gameplay)
all_frames = extractor.run_simulation(total_frames=FPS * 100)

# Extract best clips
clips = extractor.extract_clips(all_frames, num_clips=5)

# Save clips
saved_files = extractor.save_clips(clips)

print(f"\n🎉 Done! Generated {len(saved_files)} clips.")

In [ ]:
# === DISPLAY CLIPS IN NOTEBOOK ===
from IPython.display import Video, display

print("\n📺 Preview clips:")
for i, filename in enumerate(saved_files[:3]):  # Show first 3
    print(f"\nClip #{i+1}: {filename}")
    display(Video(filename, embed=True, width=480))

In [ ]:
# === DOWNLOAD ALL CLIPS ===
from google.colab import files
import zipfile

# Zip all clips
zip_filename = 'locomotio_clips.zip'
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for f in saved_files:
        zipf.write(f)

print(f"\n📦 Download all clips:")
files.download(zip_filename)

## 🎛️ Configuration Options

Adjust these parameters to generate different types of clips:

```python
# More trains = more chaos
game = HeadlessGame(num_trains=12)

# Longer simulation = more events
all_frames = extractor.run_simulation(total_frames=FPS * 300)  # 5 minutes

# More clips
clips = extractor.extract_clips(all_frames, num_clips=10)
```

### Excitement Scoring:
- **Kill**: 50 points base
- **Kill Streak**: +30 points per additional kill within 3 sec
- **Underdog Bonus**: +5 points per length difference (smaller kills bigger)
- **Leader Change**: 40 points
- **Close Call**: 30+ points based on proximity